In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

base_dir = 'c:/Users/deepa/Documents/Deeplearning CA/deep-learning-techniques-comparison-rnn-tcn'
preprocessed_dir = os.path.join(base_dir, 'preprocessed_data')

In [ ]:
with open(os.path.join(preprocessed_dir, 'har.pkl'), 'rb') as f:
    har_data = pickle.load(f)

X_train_seq = torch.FloatTensor(har_data['X_train_seq']).to(device)
X_val_seq = torch.FloatTensor(har_data['X_val_seq']).to(device)
X_test_seq = torch.FloatTensor(har_data['X_test_seq']).to(device)

y_train_seq = torch.LongTensor(har_data['y_train_seq']).to(device)
y_val_seq = torch.LongTensor(har_data['y_val_seq']).to(device)
y_test_seq = torch.LongTensor(har_data['y_test_seq']).to(device)

print(f"X_train shape: {X_train_seq.shape}")
print(f"X_val shape: {X_val_seq.shape}")
print(f"X_test shape: {X_test_seq.shape}")
print(f"y_train unique classes: {torch.unique(y_train_seq).cpu().numpy()}")
print(f"Sequence length: {X_train_seq.shape[1]}, Features: {X_train_seq.shape[2]}, Classes: {len(torch.unique(y_train_seq))}")
print(f"Device: {device}")

In [ ]:
class VanillaRNN(nn.Module):
    def __init__(self, input_size, hidden_units, num_classes):
        super(VanillaRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_units, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(hidden_units, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

seq_length = X_train_seq.shape[1]
num_features = X_train_seq.shape[2]
num_classes = len(torch.unique(y_train_seq))

hidden_units = 128
epochs = 50
batch_size = 32
learning_rate = 0.001

model = VanillaRNN(num_features, hidden_units, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(model)

In [ ]:
train_dataset = TensorDataset(X_train_seq, y_train_seq)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val_seq, y_val_seq)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []
train_accs = []
val_accs = []

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += y_batch.size(0)
        train_correct += (predicted == y_batch).sum().item()
    
    train_losses.append(train_loss / len(train_loader))
    train_accs.append(train_correct / train_total)
    
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += y_batch.size(0)
            val_correct += (predicted == y_batch).sum().item()
    
    val_losses.append(val_loss / len(val_loader))
    val_accs.append(val_correct / val_total)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accs[-1]:.4f}')

In [ ]:
model.eval()
y_pred_list = []
y_test_list = []

with torch.no_grad():
    outputs = model(X_test_seq)
    _, y_pred = torch.max(outputs, 1)
    y_pred_list = y_pred.cpu().numpy()
    y_test_list = y_test_seq.cpu().numpy()

accuracy = accuracy_score(y_test_list, y_pred_list)
f1 = f1_score(y_test_list, y_pred_list, average='weighted')

print(f"\n{'='*50}")
print(f"PyTorch Vanilla RNN - HAR Classification Results")
print(f"{'='*50}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score (weighted): {f1:.4f}")
print(f"\nConfusion Matrix:\n{confusion_matrix(y_test_list, y_pred_list)}")
print(f"\nClassification Report:\n{classification_report(y_test_list, y_pred_list)}")

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()